regression

In [1]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

# Set the parameters for the model:
params = {
    'objective': 'reg:squarederror',
    # the objective function to optimize,
    'eval_metric': 'rmse',
    # the evaluation metric to use, 
    'max_depth': 3,
    # the maximum depth of the decision trees
    'subsample': 0.5,
    # Subsample ratio of the training instances; prevent overfitting.
    'eta': 0.1
    # learning rate (step weight)
}

/home/yishai/Data/anaconda3/envs/tf_39/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [39]:
# Create some example data - random data for the inputs and outputs
N=100
X = np.random.rand(N, 5)
y = np.random.rand(N)


In [40]:
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size=0.85)

# Fit the model
# num_boost_round parameter controls the number of boosting rounds to perform.
xg_reg = xgb.train(params, xgb.DMatrix(x_train, label=y_train), num_boost_round=100)

In [41]:
y_val_pred = xg_reg.predict(xgb.DMatrix(x_val, label=y_val))
print(y_val)
print (y_val_pred)
# print accuracy score


mse = mean_squared_error(y_val_pred, y_val)
print (mse)


[0.18361392 0.63283575 0.05486231 0.80371151 0.63106665 0.03627778
 0.72004909 0.07642556 0.94889637 0.88718621 0.9683425  0.35387592
 0.90973005 0.98934819 0.5049131 ]
[ 0.38639677  0.3714008   0.38339356  0.7066531   0.51028347  0.4405099
  0.7368475   0.46929583  0.37410334  0.4954321  -0.01867209  0.42942196
  0.31609666  0.42887527  0.65310866]
0.1807799566171134


classification

In [59]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
from sklearn.datasets import load_wine
X, y = load_wine(return_X_y=True, as_frame=True)

In [60]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
xg_classify = XGBClassifier(eval_metric='mlogloss')
xg_classify.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [63]:
y_val_pred = xg_classify.predict(x_val)
accuracy = accuracy_score(y_val, y_val_pred)
accuracy

lightgbm